# --- Day 3: No Matter How You Slice It ---

The Elves managed to locate the chimney-squeeze prototype fabric for Santa's suit (thanks to someone who helpfully wrote its box IDs on the wall of the warehouse in the middle of the night). Unfortunately, anomalies are still affecting them - nobody can even agree on how to cut the fabric.

The whole piece of fabric they're working on is a very large square - at least 1000 inches on each side.

Each Elf has made a claim about which area of fabric would be ideal for Santa's suit. All claims have an ID and consist of a single rectangle with edges parallel to the edges of the fabric. Each claim's rectangle is defined as follows:

The number of inches between the left edge of the fabric and the left edge of the rectangle.
The number of inches between the top edge of the fabric and the top edge of the rectangle.
The width of the rectangle in inches.
The height of the rectangle in inches.
A claim like `#123 @ 3,2: 5x4` means that claim ID `123` specifies a rectangle 3 inches from the left edge, 2 inches from the top edge, 5 inches wide, and 4 inches tall. Visually, it claims the square inches of fabric represented by # (and ignores the square inches of fabric represented by .) in the diagram below:

```
...........
...........
...#####...
...#####...
...#####...
...#####...
...........
...........
...........
```

The problem is that many of the claims overlap, causing two or more claims to cover part of the same areas. For example, consider the following claims:

```
#1 @ 1,3: 4x4
#2 @ 3,1: 4x4
#3 @ 5,5: 2x2
```

Visually, these claim the following areas:

```
........
...2222.
...2222.
.11XX22.
.11XX22.
.111133.
.111133.
........
```

The four square inches marked with X are claimed by both 1 and 2. (Claim 3, while adjacent to the others, does not overlap either of them.)

If the Elves all proceed with their own plans, none of them will have enough fabric. How many square inches of fabric are within two or more claims?

In [21]:
import dataclasses
import itertools as it
import re

In [8]:
RECT_RE = re.compile(r'^#(?P<id_>\d+) @ (?P<left>\d+),(?P<top>\d+): (?P<width>\d+)x(?P<height>\d+)$')

In [9]:
RECT_RE.match('#1 @ 1,3: 4x4')

<re.Match object; span=(0, 13), match='#1 @ 1,3: 4x4'>

In [1]:
def test_input1():
    yield '#1 @ 1,3: 4x4'
    yield '#2 @ 3,1: 4x4'
    yield '#3 @ 5,5: 2x2'

In [2]:
def input_():
    with open('./inputs/day03/input.txt') as fp:
        yield from (line.strip() for line in fp)

In [57]:
@dataclasses.dataclass
class Rectangle:
    id_: int
    left: int
    top: int
    width: int
    height: int
    right: int = dataclasses.field(init=False)
    bottom: int = dataclasses.field(init=False)
        
    def __post_init__(self):
        self.right = self.left + self.width
        self.bottom = self.top + self.height
        
    @classmethod
    def from_str(cls, string):
        match = RECT_RE.match(string)
        if not match:
            raise RuntimeError(f'Could not match string {string:!r}')
        return cls(
            int(match.group('id_')),
            int(match.group('left')), int(match.group('top')),
            int(match.group('width')), int(match.group('height')),
        )
    
    def __hash__(self):
        return hash(self.id_)
    
    def overlap(self, other):
        return (
            ((self.left <= other.left < self.right) or (other.left <= self.left < other.right))
            and ((self.top <= other.top < self.bottom) or (other.top <= self.top < other.bottom))
        )

In [17]:
Rectangle.from_str('#1 @ 1,3: 4x4')

Rectangle(id_=1, left=1, top=3, width=4, height=4, right=5, bottom=7)

In [19]:
print(max(Rectangle.from_str(r).right for r in input_()))
print(max(Rectangle.from_str(r).bottom for r in input_()))

999
999


In [25]:
def rectangles(lines):
    return [Rectangle.from_str(r) for r in lines]

In [40]:
def count_overlap(width, height, rectangles):
    overlap_count = 0

    for column in range(width):
        for row in range(height):
            matched_count = 0

            for rect in rectangles:
                if (rect.left <= column < rect.right) and (rect.top <= row < rect.bottom):
                    matched_count += 1
                if matched_count >= 2:
                    overlap_count += 1
                    break
    
    return overlap_count

In [41]:
print(count_overlap(8, 8, rectangles(test_input1())))

4


In [42]:
print(count_overlap(1000, 1000, rectangles(input_())))

107663


# --- Part Two ---

Amidst the chaos, you notice that exactly one claim doesn't overlap by even a single square inch of fabric with any other claim. If you can somehow draw attention to it, maybe the Elves will be able to make Santa's suit after all!

For example, in the claims above, only claim 3 is intact after all claims are made.

What is the ID of the only claim that doesn't overlap?

In [69]:
def find_lonely(rectangles):
    rectangles_set = set(rectangles)
    lonely = set(rectangles)
    
    for rect1 in rectangles:            
        for rect2 in rectangles_set:
            if (
                rect2 in lonely and (rect1.id_ != rect2.id_) and rect1.overlap(rect2)
            ):
                lonely.discard(rect1)
                lonely.discard(rect2)
    
    return lonely

In [70]:
find_lonely(rectangles(test_input1()))

{Rectangle(id_=3, left=5, top=5, width=2, height=2, right=7, bottom=7)}

In [71]:
find_lonely(rectangles(input_()))

{Rectangle(id_=1166, left=126, top=200, width=10, height=11, right=136, bottom=211)}